In [1]:
import json
import os.path
import pathlib
# import bdgd_tools
import time
# from bdgd_tools import Sample, Case, Circuit, LineCode
import geopandas as gpd
import re

class JsonData:
    def __init__(self, file_name):
        """
        Inicializa a classe JsonData com o nome do arquivo de entrada.

        :param file_name: Nome do arquivo JSON de entrada.
        """
        self.data = self._read_json_file(file_name)
        self.tables = self._create_tables()

    @staticmethod
    def _read_json_file(file_name):
        """
        Lê o arquivo JSON fornecido e retorna o conteúdo como um objeto Python.

        :param file_name: Nome do arquivo JSON de entrada.
        :return: Objeto Python contendo o conteúdo do arquivo JSON.
        """
        with open(file_name, 'r', encoding='utf-8') as file:
            data = json.load(file)
        return data

    def _create_tables(self):
        """
        Cria um dicionário de tabelas a partir dos dados carregados do arquivo JSON.

        :return: Dicionário contendo informações das tabelas a serem processadas.
        """
        return {
            table_name: Table(
                table_name,
                table_data["columns"],
                table_data["type"],
                table_data["ignore_geometry"],
            )
            for table_name, table_data in self.data["configuration"][
                "tables"
            ].items()
        }

    def get_tables(self):
        """
        Retorna o dicionário de tabelas.

        :return: Dicionário contendo informações das tabelas a serem processadas.
        """
        return self.tables

    @staticmethod
    def convert_data_types(df, column_types):
        """
        Converte os tipos de dados das colunas do DataFrame fornecido.

        :param df: DataFrame a ser processado.
        :param column_types: Dicionário contendo mapeamento de colunas para tipos de dados.
        :return: DataFrame com tipos de dados convertidos.
        """
        return df.astype(column_types)

    def create_geodataframes(self, file_name, runs=1):
        """
        Cria GeoDataFrames a partir de um arquivo de entrada e coleta estatísticas.

        :param file_name: Nome do arquivo de entrada.
        :param runs: Número de vezes que cada tabela será carregada e convertida (padrão: 1).
        :return: Dicionário contendo GeoDataFrames e estatísticas.
        """
        geodataframes = {}

        for table_name, table in self.tables.items():
            load_times = []
            conversion_times = []
            gdf_converted = None

            for _ in range(runs):
                start_time = time.time()
                gdf_ = gpd.read_file(file_name, layer=table.name, include_fields=table.columns,
                                     ignore_geometry=table.ignore_geometry)
                start_conversion_time = time.time()
                gdf_converted = self.convert_data_types(gdf_, table.data_types)                    
                end_time = time.time()
 
                load_times.append(start_conversion_time - start_time)
                conversion_times.append(end_time - start_conversion_time)

            load_time_avg = sum(load_times) / len(load_times)
            conversion_time_avg = sum(conversion_times) / len(conversion_times)
            mem_usage = gdf_converted.memory_usage(index=True, deep=True).sum() / 1024 ** 2

            geodataframes[table_name] = {
                'gdf': gdf_converted,
                'memory_usage': mem_usage,
                'load_time_avg': load_time_avg,
                'conversion_time_avg': conversion_time_avg,
                'ignore_geometry': table.ignore_geometry
            }
        return geodataframes




In [2]:
import pandas as pd

class Table:
    def __init__(self, name, columns, data_types, ignore_geometry_):
        self.name = name
        self.columns = columns
        self.data_types = data_types
        self.ignore_geometry = ignore_geometry_

    def __str__(self):
        return f"Table(name={self.name}, columns={self.columns}, data_types={self.data_types}, " \
               f"ignore_geometry={self.ignore_geometry})"


In [3]:
def process_loadshape(loadshape_list):
    """
        Process a list of 96 floating point numbers and return a list of 24
        floating point numbers. The function computes the mean of every four
        numbers in the input list and normalizes the resulting list between 0
        and 1.

        Parameters
        ----------
        loadshape_list : list of float
            The input list containing 96 floating point numbers.

        Returns
        -------
        list of float
            A list containing 24 floating point numbers, which are the mean of
            every four numbers in the input list, normalized between 0 and 1.

        """
    
    medias = [sum(loadshape_list[i:i + 4]) / 4 for i in range(0, len(loadshape_list), 4)]

    

    # Calculate the minimum and maximum values in the array
    min_value = min(medias)
    max_value = max(medias)

    # Check if the range is zero
    if max_value - min_value == 0:
        # Handle the case when the range is zero (all values are the same)
        return [1 for _ in medias], medias  # Set all values to 0.5 (midpoint)

    else:
        # Normalize the array to the range [0, 1]
        return [(media - min_value) / (max_value - min_value) for media in medias], medias


def compute_load_curve(dataframe: gpd.geodataframe.GeoDataFrame):

    dataframe['loadshape'] = None
    dataframe['pot_atv'] = None

    for i in range(0,len(dataframe)):
        pot_atv_normal, pot_atv = process_loadshape(dataframe.filter(regex='^POT').loc[i,:].to_list())        # manda uma lista com os 96 valores de uma carga apenas
        
        
        dataframe.at[i,'loadshape'] = pot_atv_normal
        dataframe.at[i,'pot_atv'] = pot_atv
        
    dataframe = dataframe.loc[:, ['COD_ID', 'TIP_DIA', 'loadshape', 'pot_atv']]
    dataframe.set_index('TIP_DIA', inplace=True)

    return dataframe



In [3]:
print(os.getcwd())

c:\Users\ADERALDO\Desktop\TCC BDGB\BDGD_Dev\bdgd-tools\bdgd_tools


In [40]:
json_file_name = os.path.join(os.getcwd(), "bdgd2dss.json")

json_data = JsonData(json_file_name)

In [23]:
json_data.tables

{'CTMT': <__main__.Table at 0x2a6648aa4f0>,
 'UCMT_tab': <__main__.Table at 0x2a64d3f7ac0>}

In [41]:

json_data.get_tables()

{'CTMT': <__main__.Table at 0x1c92dc25b50>,
 'SEGCON': <__main__.Table at 0x1c92dc253d0>,
 'CRVCRG': <__main__.Table at 0x1c92dc25400>,
 'SSDMT': <__main__.Table at 0x1c92dc25a60>,
 'UNSEMT': <__main__.Table at 0x1c92dc25eb0>,
 'UNTRMT': <__main__.Table at 0x1c92dc28dc0>,
 'EQTRMT': <__main__.Table at 0x1c92dc28eb0>,
 'UNREMT': <__main__.Table at 0x1c92dc28fa0>,
 'EQRE': <__main__.Table at 0x1c92dc2ef70>,
 'SSDBT': <__main__.Table at 0x1c92dc2e6d0>,
 'RAMLIG': <__main__.Table at 0x1c92dc2e5e0>,
 'UCMT_tab': <__main__.Table at 0x1c92dc2e4c0>,
 'UCBT_tab': <__main__.Table at 0x1c92dc2e4f0>}

In [42]:
loadshape_config = json_data.data["elements"]["Loadshape"]["CRVCRG"]

In [43]:
calculated = loadshape_config.get('calculated')
print(calculated)


{'mult': ['POT_01', '-', 'POT_24']}


In [29]:

def qt_tipdia_mes(case, month):
    switch_dict = {
        "DU": {    
            '01': 21,
            '02': 20,
            '03': 22,
            '04': 19,
            '05': 22,
            '06': 21,
            '07': 21,
            '08': 23,
            '09': 21,
            '10': 20,
            '11': 20,
            '12': 22
        },
        "SA": { 
            '01': 4,
            '02': 4,
            '03': 4,
            '04': 5,
            '05': 4,
            '06': 4,
            '07': 5,
            '08': 4,
            '09': 4,
            '10': 5,
            '11': 4,
            '12': 5   
        },
        "DO": { 
            '01': 6,
            '02': 4,
            '03': 5,
            '04': 6,
            '05': 5,
            '06': 5,
            '07': 5,
            '08': 4,
            '09': 5,
            '10': 6,
            '11': 6,
            '12': 4
        }
        
    }


    
    if case in switch_dict and month in switch_dict[case]:
            return switch_dict[case][month]
    else:
        return 'Invalid case or month'

In [44]:
geodataframes = json_data.create_geodataframes(r"C:\Users\ADERALDO\Desktop\TCC BDGB\BDGD_Dev\bdgd-tools\bdgd_tools\sample\raw\aneel\muxenergia.gdb")
# geodataframes = json_data.create_geodataframes(r"C:\Users\ADERALDO\Desktop\TCC BDGB\BDGD_Dev\bdgd-tools\bdgd_tools\sample\raw\aneel\muxenergia.gdb")
# geodataframes2 = json_data.create_geodataframes(r"C:\Users\ADERALDO\Desktop\TCC BDGB\ENEL_CE_39_2021-12-31_V10_20230124-0859.gdb")

In [74]:
nom_alimentador = '2_TAP2_1'
# alimentador = "CTMT=={nom_alimentador}"
df = geodataframes["UCBT_tab"]['gdf']
var = df.query("CTMT==@nom_alimentador")
# df
var

,PAC,PN_CON,CTMT,TIP_CC,FAS_CON,TEN_FORN
2,UC14865,1284,2_TAP2_1,RES_2,BCN,15
3,UC15172,7585,2_TAP2_1,RES_2,AN,10
7,UC15176,1321,2_TAP2_1,COM_13,ABCN,15
10,UC14865,1284,2_TAP2_1,RES_2,CAN,15
12,UC20061,7553,2_TAP2_1,RES_7,ABCN,15
...,...,...,...,...,...,...
12336,UC321,1341,2_TAP2_1,RES_2,CN,10
12339,UC328,164,2_TAP2_1,RES_11,AN,10
12347,UC738,1975,2_TAP2_1,RES_2,CN,10
12350,UC8812,71,2_TAP2_1,RES_2,CN,10


In [31]:
tip = 'COM_1'
df = geodataframes["CRVCRG"]['gdf'] 
tip_dia = "DU"
df = compute_load_curve(df)
df

,COD_ID,loadshape,pot_atv
TIP_DIA,,,
SA,RUR_11,"[0.06634152074274298, 0.009284287225290873, 0....","[0.299, 0.2019, 0.3012, 0.3622, 0.1861, 0.2606..."
DO,RUR_11,"[0.06353402694866109, 0.04042299164250382, 0.3...","[0.3035, 0.2764, 0.6116, 0.2776, 0.3396, 0.229..."
DU,COM_1,"[0.17311981892876577, 0.0, 0.00573455837478886...","[457.1355, 382.729, 385.1937, 492.726, 514.909..."
SA,COM_1,"[0.3024964784933459, 0.04778334327981347, 0.0,...","[507.2172, 405.9361, 386.9361, 505.7341, 533.9..."
DO,COM_1,"[0.3966504525029378, 0.08423716948569962, 0.0,...","[512.0236, 407.8612, 379.7755, 497.1201, 508.5..."
...,...,...,...
SA,IND_12,"[0.997332750706291, 0.995126267105656, 0.22495...","[46.1763, 46.0743, 10.4711, 0.0764, 0.0807, 15..."
DO,IND_12,"[0.9992632863064426, 0.9983423941894957, 0.231...","[46.1933, 46.1508, 10.7473, 0.0807, 0.0807, 0...."
DU,IND_13,"[0.09983796132953286, 0.0, 0.00038518980794241...","[64.6877, 49.2658, 49.3253, 57.2423, 64.6452, ..."


In [17]:
tip_dia = 'DO'

In [18]:
import numpy as np

crv_dataframe_aux = df[df['COD_ID'] == f'COM_1']
values_list = [0.04585, 0.046, 0.042475, 0.04, 0.04025, 0.040175, 0.040825, 0.04115, 0.0402, 0.05, 0.052325, 0.059875, 0.059825, 0.058475, 0.044625, 0.04065, 0.03755, 0.027025, 0.01675, 0.003225, 0.003025, 0.00225, 0.00205, 0.0028]
crv_dataframe_aux.at["DO", 'pot_atv'] = values_list


# df['soma_pot'] = np.sum(np.array(df['pot_atv'].tolist()), axis=1)
crv_dataframe_aux.loc['DO','soma_pot'] = sum(values_list)
crv_dataframe_aux.loc['DU','soma_pot'] = 1.27
crv_dataframe_aux.loc['SA','soma_pot'] = 1.07


pot_classe = crv_dataframe_aux['soma_pot'].sum()

crv_dataframe_aux['prop'] = crv_dataframe_aux['soma_pot'] / pot_classe

for index, row in crv_dataframe_aux.iterrows():
    crv_dataframe_aux.loc[index, "prop_pot_tipdia_mes"] = row["prop"]*qt_tipdia_mes(index,'01')


prop_pot_mens_mes = crv_dataframe_aux["prop_pot_tipdia_mes"][tip_dia]/(crv_dataframe_aux["prop_pot_tipdia_mes"].sum())

# crv_dataframe_aux
# pot_classe
prop_pot_mens_mes

pot_atv_media = crv_dataframe_aux["soma_pot"][tip_dia]/24
pot_atv_max = max(crv_dataframe_aux["pot_atv"][tip_dia])
fc = pot_atv_media/pot_atv_max

demand_max = 1.5740 * (prop_pot_mens_mes*1000)/(qt_tipdia_mes(tip_dia, '01')*24*fc)

demand_max

C:\Users\ADERALDO\AppData\Local\Temp\ipykernel_15112\4279639142.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crv_dataframe_aux.loc['DO','soma_pot'] = sum(values_list)
C:\Users\ADERALDO\AppData\Local\Temp\ipykernel_15112\4279639142.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crv_dataframe_aux['prop'] = crv_dataframe_aux['soma_pot'] / pot_classe
C:\Users\ADERALDO\AppData\Local\Temp\ipykernel_15112\4279639142.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

2.6197418987192234

In [20]:
meses = [f"{mes:02d}" for mes in range(1, 13)]
meses

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

In [23]:
class Person:
    def __init__(self, name):
        self.name = name

person = Person("Alice")
name = getattr(person, 'name')
print(name)  # This will print 'Alice'


Alice


In [11]:

df["soma_pot"] = None
for index, row in df.iterrows(): 
    df.loc[index, "soma_pot"] = sum(row["loadshape"])

df

,COD_ID,loadshape,soma_pot
TIP_DIA,,,
DU,COM_1,"[0.17311981892876577, 0.0, 0.00573455837478886...",15.555418
SA,COM_1,"[0.3024964784933459, 0.04778334327981347, 0.0,...",16.955455
DO,COM_1,"[0.3966504525029378, 0.08423716948569962, 0.0,...",17.438365


In [52]:
pot_classe = df['soma_pot'].sum()
df['prop'] = df['soma_pot'] / pot_classe
for index, row in df.iterrows():
        df.loc[index, "prop_pot_tipdia_mes"] = row["prop"]*qt_tipdia_mes(index,"01")
prop_pot_mens_mes = df["prop_pot_tipdia_mes"][tip_dia]/df["prop_pot_tipdia_mes"].sum()

df



,COD_ID,loadshape,soma_pot,prop,prop_pot_tipdia_mes
TIP_DIA,,,,,
DU,COM_1,"[0.17311981892876577, 0.0, 0.00573455837478886...",15.555418,0.311425,6.539915
SA,COM_1,"[0.3024964784933459, 0.04778334327981347, 0.0,...",16.955455,0.339454,1.357815
DO,COM_1,"[0.3966504525029378, 0.08423716948569962, 0.0,...",17.438365,0.349122,2.094730


In [54]:
pot_atv_media = df["soma_pot"][tip_dia]/24
pot_atv_media = pot_atv_media/24
pot_atv_max = max(df["loadshape"][tip_dia])
fc = pot_atv_media/pot_atv_max

demand_max = demand_max = 833 * (prop_pot_mens_mes*1000)/(qt_tipdia_mes(tip_dia, "01")*24*fc)

print(demand_max)



40054.834341003276


In [35]:

def qt_tipdia_mes(case, month):
    switch_dict = {
        "DU": {    
            '01': 21,
            '02': 20,
            '03': 22,
            '04': 19,
            '05': 22,
            '06': 21,
            '07': 21,
            '08': 23,
            '09': 21,
            '10': 20,
            '11': 20,
            '12': 22
        },
        "SA": { 
            '01': 4,
            '02': 4,
            '03': 4,
            '04': 5,
            '05': 4,
            '06': 4,
            '07': 5,
            '08': 4,
            '09': 4,
            '10': 5,
            '11': 4,
            '12': 5   
        },
        "DO": { 
            '01': 6,
            '02': 4,
            '03': 5,
            '04': 6,
            '05': 5,
            '06': 5,
            '07': 5,
            '08': 4,
            '09': 5,
            '10': 6,
            '11': 6,
            '12': 4
        }
        
    }


    
    if case in switch_dict and month in switch_dict[case]:
            return switch_dict[case][month]
    else:
        return 'Invalid case or month'

In [66]:
energia: int = 0
energia = "2"

In [67]:
type(energia)

str

In [61]:
qt_tipdia_mes("DU", "01")

21

In [52]:
df

,COD_ID,loadshape,soma_pot
TIP_DIA,,,
DU,COM_1,"[0.17311981892876577, 0.0, 0.00573455837478886...",15.555418
SA,COM_1,"[0.3024964784933459, 0.04778334327981347, 0.0,...",16.955455
DO,COM_1,"[0.3966504525029378, 0.08423716948569962, 0.0,...",17.438365


In [27]:
lista = [1,2,3,4,5]
sum(lista)

15

In [41]:
df = df.loc[:, ['COD_ID', 'TIP_DIA']]
df.loc[1,"dedo"] = "cu"
df

,COD_ID,TIP_DIA,dedo
2,COM_1,DU,NaN
1,NaN,NaN,cu


In [78]:
df = df[(df['COD_ID'] == "RUR_1") ]
# df["soma"] = [1,2,3]
# df["soma"]
df["soma"] = [1, 2, 3]
df = df.reset_index(drop=True)
# sum(df["soma"][0])
sum(df["soma"])

6

In [9]:
merged_dfs = pd.merge( geodataframes["EQRE"]['gdf'], geodataframes["UNREMT"]['gdf'],left_on='UN_RE', right_on='COD_ID', how='inner')
merged_dfs

,COD_ID_x,POT_NOM,TEN_REG,LIG_FAS_P,LIG_FAS_S,PER_FER,PER_TOT,XHL,UN_RE,COD_ID_y,FAS_CON,PAC_1,PAC_2,CTMT,TIP_REGU
0,1,23,1.05,AB,AB,630.73,2049.0,2.0,2_3002,2_3002,ABC,RT24361,24371,2_TAP2_1,DF
1,11,23,1.05,BC,BC,630.73,2049.0,2.0,2_3002,2_3002,ABC,RT24361,24371,2_TAP2_1,DF
2,12,23,1.05,CA,CA,630.73,2049.0,2.0,2_3002,2_3002,ABC,RT24361,24371,2_TAP2_1,DF
3,10,42,1.05,AB,AB,1600.00,5200.0,4.0,1_3000,1_3000,ABC,RT4711,82261,1_TAP2_1,DF
4,2,42,1.05,BC,BC,1600.00,5200.0,4.0,1_3000,1_3000,ABC,RT4711,82261,1_TAP2_1,DF
5,9,42,1.05,CA,CA,1600.00,5200.0,4.0,1_3000,1_3000,ABC,RT4711,82261,1_TAP2_1,DF
6,13,42,1.05,AB,AB,1600.00,5200.0,3.7,1_3001,1_3001,ABC,RT27361,82601,2_TAP2_1,DF
7,14,42,1.05,BC,BC,1600.00,5200.0,3.7,1_3001,1_3001,ABC,RT27361,82601,2_TAP2_1,DF
8,15,42,1.05,CA,CA,1600.00,5200.0,3.7,1_3001,1_3001,ABC,RT27361,82601,2_TAP2_1,DF


In [1]:
string = "teste"

In [3]:
foi = string[2] + string[3]
foi

'st'

In [71]:
inner_merge_entities(geodataframes["EQRE"]['gdf'],geodataframes["UNREMT"]['gdf'])

,POT_NOM,TEN_REG,LIG_FAS_P,LIG_FAS_S,PER_FER,PER_TOT,XHL,UN_RE,COD_ID,FAS_CON,PAC_1,PAC_2,CTMT,TIP_REGU
0,23,1.05,AB,AB,630.73,2049.0,2.0,2_3002,2_3002,ABC,RT24361,24371,2_TAP2_1,DF
1,23,1.05,BC,BC,630.73,2049.0,2.0,2_3002,2_3002,ABC,RT24361,24371,2_TAP2_1,DF
2,23,1.05,CA,CA,630.73,2049.0,2.0,2_3002,2_3002,ABC,RT24361,24371,2_TAP2_1,DF
3,42,1.05,AB,AB,1600.00,5200.0,4.0,1_3000,1_3000,ABC,RT4711,82261,1_TAP2_1,DF
4,42,1.05,BC,BC,1600.00,5200.0,4.0,1_3000,1_3000,ABC,RT4711,82261,1_TAP2_1,DF
5,42,1.05,CA,CA,1600.00,5200.0,4.0,1_3000,1_3000,ABC,RT4711,82261,1_TAP2_1,DF
6,42,1.05,AB,AB,1600.00,5200.0,3.7,1_3001,1_3001,ABC,RT27361,82601,2_TAP2_1,DF
7,42,1.05,BC,BC,1600.00,5200.0,3.7,1_3001,1_3001,ABC,RT27361,82601,2_TAP2_1,DF
8,42,1.05,CA,CA,1600.00,5200.0,3.7,1_3001,1_3001,ABC,RT27361,82601,2_TAP2_1,DF


In [74]:
current_directory = os.getcwd()  # Get the current working directory
parent_directory = os.path.dirname(current_directory)  # Get the parent directory
parent_directory

'c:\\Users\\ADERALDO\\Desktop\\TCC BDGB\\BDGD_Dev\\bdgd-tools'

In [86]:
def create_txt_file_from_strings(string_list, file_name):
    current_directory = os.getcwd()  # Get the current working directory
    parent_directory = os.path.dirname(current_directory)  # Get the parent directory
    output_directory= os.path.join(parent_directory, "output")

    caminho = os.path.join(output_directory, file_name)

    try:
        with open(caminho, "w") as file:
            for string in string_list:
                file.write(string + "\n")
        print(f'O arquivo {file_name} foi gerado')
    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Example usage:
list_of_strings = [
    "This is the first line.",
    "This is the second line.",
    "And this is the third line."
]
create_txt_file_from_strings(list_of_strings, "output.txt")


3 strings were written to output.txt


In [3]:
dataframe1 = geodataframes['UNTRMT']['gdf']

NameError: name 'geodataframes' is not defined

In [99]:
dataframe2 = geodataframes['EQTRMT']['gdf']


In [2]:
for i in range(1, 5):
    print(i)


1
2
3
4


In [1]:
f"New \"Linecode.{self.linecode}\" nphases={self.nphases} " \
f"basefreq={self.basefreq} r1=\"{self.r1}\" x1={self.x1} " \
f"units={self.units} normamps={self.normamps}"

f'New "Linecode.{linecode_num}_{nphases_value}" nphases={nphases_value}'

NameError: name 'self' is not defined

In [100]:
merged_dfs= dataframe2.join(dataframe1, lsuffix='_left')
duplicated_columns = [col for col in merged_dfs.columns if '_left' in col]
merged_dfs.drop(columns=duplicated_columns, inplace=True)

merged_dfs

,FAS_CON,TEN_PRI,TEN_SEC,TEN_TER,COD_ID,PAC_1,PAC_2,PAC_3,FAS_CON_P,FAS_CON_S,FAS_CON_T,TAP,POT_NOM,PER_FER,PER_TOT,CTMT,TIP_TRAFO
0,ABC,49,15,0,1_165,86851,ET1_165,0,ABC,ABCN,0,1.045,45.0,215.0,1049.0,1_TAP2_1,T
1,ABC,49,15,0,1_14,13511,ET1_14,0,ABC,ABCN,0,1.045,75.0,334.0,1661.0,1_TAP2_1,T
2,ABC,49,15,0,1_217,86751,ET1_217,0,ABC,ABCN,0,1.045,112.5,430.0,2541.0,2_TAP2_1,T
3,ABC,49,15,0,1_110,25191,ET1_110,0,ABC,ABCN,0,1.045,45.0,215.0,1049.0,2_TAP2_1,T
4,B,39,17,0,1_200,82411,ET1_200,0,ABC,ABCN,0,1.045,45.0,215.0,1049.0,2_TAP2_1,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,ABC,49,15,0,1_47,8371,ET1_47,0,ABC,ABCN,0,1.045,45.0,215.0,1049.0,1_TAP2_1,T
247,ABC,49,15,0,1_127,11131,ET1_127,0,ABC,ABCN,0,1.045,150.0,462.0,2335.0,1_TAP2_1,T
248,ABC,49,15,0,1_67,75951,ET1_67,0,ABC,ABCN,0,1.045,300.0,972.0,4483.0,2_TAP2_1,T
249,ABC,49,15,0,1_83,13481,ET1_83,0,ABC,ABCN,0,1.045,45.0,215.0,1049.0,1_TAP2_1,T


In [69]:
merged_dfs = pd.merge( geodataframes["EQRE"]['gdf'], geodataframes["UNREMT"]['gdf'],left_on='UN_RE', right_on='COD_ID', how='inner')


def inner_merge_entities(entity1_df, enetity2_df):

    merged_dfs = pd.merge( entity1_df, enetity2_df,left_on='UN_RE', right_on='COD_ID', how='inner')

    for column in merged_dfs.columns:
        if column.endswith('_x'):
            merged_dfs.drop(columns=column, inplace=True)
        elif column.endswith('_y'):
            new_column_name = column[:-2]  # Remove the '_df2' suffix
            merged_dfs.rename(columns={column: new_column_name}, inplace=True)

    return merged_dfs
merged_dfs

,COD_ID_x,POT_NOM,TEN_REG,LIG_FAS_P,LIG_FAS_S,PER_FER,PER_TOT,XHL,UN_RE,COD_ID_y,FAS_CON,PAC_1,PAC_2,CTMT,TIP_REGU
0,1,23,1.05,AB,AB,630.73,2049.0,2.0,2_3002,2_3002,ABC,RT24361,24371,2_TAP2_1,DF
1,11,23,1.05,BC,BC,630.73,2049.0,2.0,2_3002,2_3002,ABC,RT24361,24371,2_TAP2_1,DF
2,12,23,1.05,CA,CA,630.73,2049.0,2.0,2_3002,2_3002,ABC,RT24361,24371,2_TAP2_1,DF
3,10,42,1.05,AB,AB,1600.00,5200.0,4.0,1_3000,1_3000,ABC,RT4711,82261,1_TAP2_1,DF
4,2,42,1.05,BC,BC,1600.00,5200.0,4.0,1_3000,1_3000,ABC,RT4711,82261,1_TAP2_1,DF
5,9,42,1.05,CA,CA,1600.00,5200.0,4.0,1_3000,1_3000,ABC,RT4711,82261,1_TAP2_1,DF
6,13,42,1.05,AB,AB,1600.00,5200.0,3.7,1_3001,1_3001,ABC,RT27361,82601,2_TAP2_1,DF
7,14,42,1.05,BC,BC,1600.00,5200.0,3.7,1_3001,1_3001,ABC,RT27361,82601,2_TAP2_1,DF
8,15,42,1.05,CA,CA,1600.00,5200.0,3.7,1_3001,1_3001,ABC,RT27361,82601,2_TAP2_1,DF


In [52]:
geodataframes["EQRE"]['gdf']

,COD_ID,POT_NOM,TEN_REG,LIG_FAS_P,LIG_FAS_S,PER_FER,PER_TOT,XHL,UN_RE
0,1,23,1.05,AB,AB,630.73,2049.0,2.0,2_3002
1,10,42,1.05,AB,AB,1600.00,5200.0,4.0,1_3000
2,11,23,1.05,BC,BC,630.73,2049.0,2.0,2_3002
3,12,23,1.05,CA,CA,630.73,2049.0,2.0,2_3002
4,13,42,1.05,AB,AB,1600.00,5200.0,3.7,1_3001
5,14,42,1.05,BC,BC,1600.00,5200.0,3.7,1_3001
6,15,42,1.05,CA,CA,1600.00,5200.0,3.7,1_3001
7,2,42,1.05,BC,BC,1600.00,5200.0,4.0,1_3000
8,9,42,1.05,CA,CA,1600.00,5200.0,4.0,1_3000


In [14]:
geodataframes["UNREMT"]['gdf']

,COD_ID,FAS_CON,PAC_1,PAC_2,CTMT,TIP_REGU
0,1_3000,ABC,RT4711,82261,1_TAP2_1,DF
1,2_3002,ABC,RT24361,24371,2_TAP2_1,DF
2,1_3001,ABC,RT27361,82601,2_TAP2_1,DF


In [169]:
should_include_bus3 = "priquito" # Replace with your condition for including self.bus3
bus3="deu deu"
bus3_nodes="deu nao deu 3"
priquitao = "priquito"
formatted_string = (

    f'buses=["' +
    (f' "{bus3}.{bus3_nodes}"' if should_include_bus3 == priquitao else '') +
    f'] conns='
)

print(formatted_string)

buses=[" "deu deu.deu nao deu 3"] conns=


In [150]:
should_include_bus3 = True  # Replace with your condition for including self.bus3

formatted_string = (
    f'New "Transformer.TRF_{self.transformer}" phases={self.phases} '
    f'windings={self.windings} '
    f'buses=["{self.bus1}.{self.bus1_nodes}" "{self.bus2}.{self.bus2_nodes}"' +
    (f' "{self.bus3}.{self.bus3_nodes}"' if should_include_bus3 else '') +
    f'] conns=[{self.conn_p} {self.conn_s} {self.conn_t}] kvs=[{self.kvs}] kvas=[{self.kvas}]'
    f' %loadloss={self.loadloss:.3f} %noloadloss={self.noloadloss:.3f}'
)

print(formatted_string)

1.8533333333333335

In [171]:
word = "Hello"
repetition_count = 5
result_string = ''.join([word for _ in range(repetition_count)])


In [1]:
word

NameError: name 'word' is not defined

In [177]:
''.join([f'{your_variable}' for _ in range(repetition_count)])

SyntaxError: invalid syntax (106891323.py, line 1)

In [144]:
df1=geodataframes['EQTRMT']['gdf']
df1

,COD_ID,POT_NOM,FAS_CON,TEN_PRI,TEN_SEC,TEN_TER,PER_FER,PER_TOT
0,35,13,ABC,49,15,0,215.0,1049.0
1,304,30,ABC,49,15,0,765.0,3465.0
2,42,24,ABC,49,15,0,462.0,2335.0
3,45,13,ABC,49,15,0,215.0,1049.0
4,164,3,B,39,17,0,53.0,268.0
...,...,...,...,...,...,...,...,...
246,261,16,ABC,49,15,0,334.0,1661.0
247,260,20,ABC,49,15,0,430.0,2541.0
248,262,4,ABC,49,15,0,110.0,480.0
249,263,16,ABC,49,15,0,334.0,1661.0


In [146]:
df2=geodataframes['UNTRMT']['gdf']["POT_NOM"]
df2

0        45.0
1        75.0
2       112.5
3        45.0
4        45.0
        ...  
246      45.0
247     150.0
248     300.0
249      45.0
250    1500.0
Name: POT_NOM, Length: 251, dtype: float64

In [120]:
merged_df = df1.join(df2, lsuffix='_left')
duplicated_columns = [col for col in merged_df.columns if '_left' in col]
merged_df.drop(columns=duplicated_columns, inplace=True)


merged_df

,FAS_CON,TEN_PRI,TEN_SEC,TEN_TER,COD_ID,PAC_1,PAC_2,PAC_3,FAS_CON_P,FAS_CON_S,FAS_CON_T,TAP,POT_NOM,PER_FER,PER_TOT,CTMT
0,ABC,49,15,0,EK0034,3862749138472183,270702806TR,0,ABC,ABCN,0,1.00,112.5,390.0,1890.0,ESB01S3
1,ABC,49,15,0,JGR01T6,5471010738449464,168462689TR,0,ABC,ABCN,0,1.00,45.0,195.0,945.0,BFG01N5
2,ABC,49,15,0,MK4539,6950215939223979,24340374TR,0,ABC,ABCN,0,1.00,15.0,85.0,410.0,VRZ01P5
3,A,49,15,0,TAA0001,4121579138893217,20252248TR,0,A,AN,0,1.00,10.0,50.0,245.0,GRM01M4
4,A,49,15,0,TAA0002,4120559238891748,20252253TR,0,A,AN,0,1.00,10.0,50.0,245.0,GRM01M4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156956,ABC,49,15,0,99W1850,3798000538610632,63531303TR,0,ABC,ABCN,0,1.05,30.0,150.0,695.0,BJD01L6
156957,ABC,49,15,0,99W2276,3798500938614203,584373TR,0,ABC,ABCN,0,1.05,45.0,195.0,945.0,BJD01L6
156958,ABC,49,15,0,99W2541,3798743838616632,584378TR,0,ABC,ABCN,0,1.05,150.0,485.0,2335.0,BJD01L6
156959,ABC,49,15,0,99W2960,3695348638581558,434027TR,0,ABC,ABCN,0,1.05,113.0,390.0,1890.0,BCR01C4


In [89]:
loadshape_list = geodataframes['CRVCRG']['gdf'].filter(regex='^POT').loc[20,:].to_list()
loadshape_list

[0.1692,
 0.1692,
 0.1692,
 0.1692,
 0.167,
 0.167,
 0.167,
 0.167,
 0.1568,
 0.1568,
 0.1568,
 0.1568,
 0.1128,
 0.1128,
 0.1128,
 0.1128,
 0.1861,
 0.1861,
 0.1861,
 0.1861,
 0.2053,
 0.2053,
 0.2053,
 0.2053,
 0.3216,
 0.3216,
 0.3216,
 0.3216,
 0.1196,
 0.1196,
 0.1196,
 0.1196,
 0.2516,
 0.2516,
 0.2516,
 0.2516,
 0.2674,
 0.2674,
 0.2674,
 0.2674,
 0.2042,
 0.2042,
 0.2042,
 0.2042,
 0.2189,
 0.2189,
 0.2189,
 0.2189,
 0.1591,
 0.1591,
 0.1591,
 0.1591,
 0.2561,
 0.2561,
 0.2561,
 0.2561,
 0.2369,
 0.2369,
 0.2369,
 0.2369,
 0.1467,
 0.1467,
 0.1467,
 0.1467,
 0.2019,
 0.2019,
 0.2019,
 0.2019,
 0.2482,
 0.2482,
 0.2482,
 0.2482,
 0.29,
 0.29,
 0.29,
 0.29,
 0.6759,
 0.6759,
 0.6759,
 0.6759,
 0.3103,
 0.3103,
 0.3103,
 0.3103,
 0.2189,
 0.2189,
 0.2189,
 0.2189,
 0.2031,
 0.2031,
 0.2031,
 0.2031,
 0.1579,
 0.1579,
 0.1579,
 0.1579]

In [42]:
import pandas as np

In [80]:
medias = [sum(loadshape_list[i:i + 4]) / 4 for i in range(0, len(loadshape_list), 4)]
medias = np.array(medias)
mean = medias.mean()
std = medias.std()
saida = [(abs(x - mean)) / std if std != 0 else 0 for x in medias]

print(type(saida))
print(saida)

<class 'list'>
[0.7717447069443004, 0.9757064957399044, 0.7671235335627316, 0.6389909988919608, 1.0088949227529893, 0.8524051877862282, 0.3331533423663176, 2.2032406841083665, 0.33754695796773326, 0.048303514948632775, 0.7429498955326311, 0.5110685175567159, 0.2667764883401479, 0.776575933661395, 0.44700225022133044, 0.6053649607631969, 1.4422174494981983, 2.56579274122599, 0.6101961874802915, 0.4423810768397617, 1.354625208583917, 0.6744900125958301, 0.8001019072402906, 0.6438222256090554]


In [95]:
import numpy as np

medias = [sum(loadshape_list[i:i + 4]) / 4 for i in range(0, len(loadshape_list), 4)]

medias = np.array(medias)

# Calculate the minimum and maximum values in the array
min_value = medias.min()
max_value = medias.max()

# Check if the range is zero
if max_value - min_value == 0:
    # Handle the case when the range is zero (all values are the same)
    normalized_data = np.full(medias.shape, 0.5)  # Set all values to 0.5 (midpoint)

else:
    # Normalize the array to the range [0, 1]
    normalized_data = (medias - min_value) / (max_value - min_value)

print(normalized_data)


[0.10015983 0.09625289 0.07813887 0.         0.13017226 0.16426922
 0.37080448 0.01207601 0.24649263 0.27455159 0.16231575 0.18842124
 0.08222341 0.25448411 0.22038714 0.06020245 0.15823122 0.24045463
 0.31468656 1.         0.35073699 0.18842124 0.16036228 0.08009235]


In [78]:
loadshape_list

[0.299,
 0.299,
 0.299,
 0.299,
 0.2019,
 0.2019,
 0.2019,
 0.2019,
 0.3012,
 0.3012,
 0.3012,
 0.3012,
 0.3622,
 0.3622,
 0.3622,
 0.3622,
 0.1861,
 0.1861,
 0.1861,
 0.1861,
 0.2606,
 0.2606,
 0.2606,
 0.2606,
 0.5078,
 0.5078,
 0.5078,
 0.5078,
 1.7153,
 1.7153,
 1.7153,
 1.7153,
 0.8271,
 0.8271,
 0.8271,
 0.8271,
 0.6894,
 0.6894,
 0.6894,
 0.6894,
 1.0201,
 1.0201,
 1.0201,
 1.0201,
 0.4231,
 0.4231,
 0.4231,
 0.4231,
 0.5394,
 0.5394,
 0.5394,
 0.5394,
 0.2967,
 0.2967,
 0.2967,
 0.2967,
 0.4536,
 0.4536,
 0.4536,
 0.4536,
 0.9546,
 0.9546,
 0.9546,
 0.9546,
 1.353,
 1.353,
 1.353,
 1.353,
 1.8879,
 1.8879,
 1.8879,
 1.8879,
 0.9569,
 0.9569,
 0.9569,
 0.9569,
 0.4558,
 0.4558,
 0.4558,
 0.4558,
 1.3113,
 1.3113,
 1.3113,
 1.3113,
 0.3453,
 0.3453,
 0.3453,
 0.3453,
 0.2855,
 0.2855,
 0.2855,
 0.2855,
 0.3599,
 0.3599,
 0.3599,
 0.3599]

In [ ]:
case = Case()
case.dfs = geodataframes
case.dfs

In [27]:
circuit_config = json_data.data['elements']['Circuit']['CTMT']

In [89]:
# -*- encoding: utf-8 -*-
"""
 * Project Name: bdgd_tests
 * Created by eniocc
 * Date: 08/03/2023
 * Time: 23:09
 *
 * Edited by: eniocc
 * Date: 21/03/2023
 * Time: 22:42
"""
# Não remover a linha de importação abaixo
from typing import Any, List
import geopandas as gpd
from tqdm import tqdm

from bdgd_tools.model.Converter import convert_tten

from dataclasses import dataclass


@dataclass
class Circuit:
    _arquivo: str = ""
    _circuit: str = ""
    _basekv: float = 0.0
    _pu: float = 0.0
    _bus1: str = ""
    _r1: float = 0.0000
    _x1: float = 0.0001

    @property
    def circuit(self) -> str:
        return self._circuit

    @circuit.setter
    def circuit(self, value):
        self._circuit = f"Circuit.{value}"

    @property
    def arquivo(self) -> str:
        return self._arquivo

    @arquivo.setter
    def arquivo(self, value):
        self._arquivo = value

    @property
    def basekv(self) -> float:
        return self._basekv

    @basekv.setter
    def basekv(self, value):
        self._basekv = value

    @property
    def pu(self) -> float:
        return self._pu

    @pu.setter
    def pu(self, value):
        self._pu = value

    @property
    def bus1(self) -> str:
        return self._bus1

    @bus1.setter
    def bus1(self, value):
        self._bus1 = value

    @property
    def r1(self) -> float:
        return self._r1

    @r1.setter
    def r1(self, value):
        self._r1 = value

    @property
    def x1(self) -> float:
        return self._x1

    @x1.setter
    def x1(self, value):
        self._x1 = value

    def __repr__(self):
        return f"New \"Circuit.{self.circuit}\" basekv={self.basekv} pu={self.pu} " \
               f"bus1=\"{self.bus1}\" r1={self.r1} x1={self.x1}"

    @staticmethod
    def _process_static(circuit_, value):
        """
        Static method to process the static configuration for a Circuit object.

        Args:
            circuit_ (object): A Circuit object being updated.
            value (dict): A dictionary containing the static configuration.

        This method processes the static configuration by iterating through the
        key-value pairs of the 'value' dictionary and directly setting the
        corresponding attribute on the Circuit object with the static value.
        """
        for static_key, static_value in value.items():
            setattr(circuit_, f"_{static_key}", static_value)

    @staticmethod
    def _process_direct_mapping(circuit_, value, row):
        """
        Static method to process the direct mapping configuration for a Circuit object.

        Args:
            circuit_ (object): A Circuit object being updated.
            value (dict): A dictionary containing the direct mapping configuration.
            row (pd.Series): A row from the GeoDataFrame containing circuit-related data.

        This method processes the direct mapping configuration by iterating through the
        key-value pairs of the 'value' dictionary and directly setting the corresponding
        attribute on the Circuit object using the value from the row.
        """
        for mapping_key, mapping_value in value.items():
            setattr(circuit_, f"_{mapping_key}", row[mapping_value])

    @staticmethod
    def _process_indirect_mapping(circuit_, value, row):
        """
        Static method to process the indirect mapping configuration for a Circuit object.

        Args:
            circuit_ (object): A Circuit object being updated.
            value (dict): A dictionary containing the indirect mapping configuration.
            row (pd.Series): A row from the GeoDataFrame containing circuit-related data.

        This method processes the indirect mapping configuration by iterating through the
        key-value pairs of the 'value' dictionary. If the value is a list, it treats the
        first element as a parameter name and the second element as a function name. The
        method then retrieves the parameter value from the row and calls the specified
        function with that parameter value. The result is then set as an attribute on the
        Circuit object.

        If the value is not a list, the method directly sets the corresponding attribute on
        the Circuit object using the value from the row.
        """

        print ("comecando o indirect mapping\n")
        print(value)
        print("\n")
        print(row) 
        
        for mapping_key, mapping_value in value.items():
            if isinstance(mapping_value, list):         #checa se é uma lista, ou seja, se tem uma funcao de conversao atrelada
                param_name, function_name = mapping_value   #pega os valores da lista como nome do parametro e nome da funcao
                print(param_name, function_name)            
                function_ = globals()[function_name]     #transforma numa função        
                param_value = row[param_name]            #pega valor para conversao 
                setattr(circuit_, f"_{mapping_key}", function_(str(param_value)))    # objeto, key (bakekv), valor convertido
                print(type(param_value))
            else:
                setattr(circuit_, f"_{mapping_key}", row[mapping_value])        # caso contrario, so coloca o valor la presente na linha




In [ ]:

dataframe = case.dfs['CTMT']['gdf']

progress_bar = tqdm(dataframe.iterrows(), total=len(dataframe), desc="Circuit", unit=" circuits", ncols=100)
for _, row in progress_bar:
    circuit_ = Circuit()
    for key, value in circuit_config.items():
        print(key, value)
        if key == "direct_mapping":
            Circuit._process_direct_mapping(circuit_, value, row)
        elif key == "indirect_mapping":
            print("ta aqui \n")
            Circuit._process_indirect_mapping(circuit_, value, row)

        elif key == "static":
            Circuit._process_static(circuit_, value)



In [91]:
# -*- encoding: utf-8 -*-
"""
 * Project Name: main.py
 * Created by eniocc
 * Date: 21/03/2023
 * Time: 23:53
 *
 * Edited by: eniocc
 * Date: 21/03/2023
 * Time: 23:53
"""
# Não remover a linha de importação abaixo
import copy
import re
from typing import Any

import geopandas as gpd
from tqdm import tqdm

from bdgd_tools.model.Converter import convert_tfascon_phases
#from bdgd_tools.model.Converter import convert_tten

from dataclasses import dataclass


@dataclass
class Line:
    
    _units: str = "km"
    _bus1: str = ""
    _bus2: str = ""
    _line: str = ""
    _linecode: str = ""
    _phases: int = 0                             # ver como funciona no circuit.py visto que isso eh um metodo indireto [FAS_CON TFASCON]
    _length: float = 0.0

    _c0: float = 0.0
    _c1: float = 0.0
    _r0: float = 0.001
    _r1: float = 0.001
    _switch: str = "T"
    _x0: float = 0.0
    _x1: float = 0.0

    @property
    def units(self):
        return self._units

    @units.setter
    def units(self, value: str):
        self._units = value

    @property
    def bus1(self):
        return self._bus1

    @bus1.setter
    def bus1(self, value: str):
        self._bus1 = value

    @property
    def bus2(self):
        return self._bus2

    @bus2.setter
    def bus2(self, value: str):
        self._bus2 = value

    @property
    def line(self):
        return self._line

    @line.setter
    def line(self, value: str):
        self._line = value

    @property
    def linecode(self):
        return self._linecode

    @linecode.setter
    def linecode(self, value: str):
        self._linecode = value

    @property
    def phases(self):
        return self._phases

    @phases.setter
    def phases(self, value: int):
        self._phases = value

    @property
    def length(self):
        return self._length

    @length.setter
    def length(self, value: float):
        self._length = value

    @property
    def c0(self):
        return self._c0

    @c0.setter
    def c0(self, value: float):
        self._c0 = value

    @property
    def c1(self):
        return self._c1

    @c1.setter
    def c1(self, value: float):
        self._c1 = value

    @property
    def r0(self):
        return self._r0

    @r0.setter
    def r0(self, value: float):
        self._r0 = value

    @property
    def r1(self):
        return self._r1

    @r1.setter
    def r1(self, value: float):
        self._r1 = value

    @property
    def switch(self):
        return self._switch

    @switch.setter
    def switch(self, value: str):
        self._switch = value

    @property
    def x0(self):
        return self._x0

    @x0.setter
    def x0(self, value: float):
        self._x0 = value

    @property
    def x1(self):
        return self._x1

    @x1.setter
    def x1(self, value: float):
        self._x1 = value



    def full_string(self) -> str:
        return  f'New "Line.SMT_{self.line}" phases={self.phases} ' \
            f'bus1="{self.bus1}.1.2.3" bus2="{self.bus2}.1.2.3" ' \
            f'linecode="{self.linecode}_{self.phases}" length={self.length:.5f} ' \
            f'units={self.units}'

    def __repr__(self):
        return  f'New "Line.SMT_{self.line}" phases={self.phases} ' \
                f'bus1="{self.bus1}.1.2.3" bus2="{self.bus2}.1.2.3" ' \
                f'linecode="{self.linecode}_{self.phases}" length={self.length:.5f} ' \
                f'units={self.units}'

    @staticmethod
    def _rename_line_string(input_str: str) -> str:
            
            """
            This function re-writes the string identfying key places by specified parameters and insering caracteres.  
            Precisa receber um argumento para definir SMT ou outra entidade. 

            Args: input_str, datagrame para identificar a entidade

            Returns: string modified

            In this case, it should modify the names of line, bus1, bus2 and linecode. 

            """

            pattern = r'New "Line.([\dA-Za-z]+)" phases=(\d+) bus1="([\dA-Za-z]+)" bus2="([\dA-Za-z]+)" linecode="(\d+)"'   #linecode=(\d+) length=(\d+) units=(\d+)' # bus1="86761" bus2="1CF155" linecode="56"'

            def repl(match):
                line_num = match.group(1)
                phases = match.group(2)
                bus1 = match.group(3)     
                bus2 = match.group(4)       
                linecode = match.group(5)
                return f'New "Line.SMT_{line_num}" phases={phases} bus1="{bus1}.1.2.3" bus2="{bus2}.1.2.3" linecode="{linecode}_{phases}"'

            return re.sub(pattern, repl, input_str)
    
    @staticmethod
    def _process_static(line_, value):
        """
        Static method to process the static configuration for a Circuit object.

        Args:
            line_ (object): A Circuit object being updated.
            value (dict): A dictionary containing the static configuration.

        This method processes the static configuration by iterating through the
        key-value pairs of the 'value' dictionary and directly setting the
        corresponding attribute on the Circuit object with the static value.
        """
        for static_key, static_value in value.items():
            setattr(line_, f"_{static_key}", static_value)

    @staticmethod
    def _process_direct_mapping(line_, value, row):
        """
        Static method to process the direct mapping configuration for a Circuit object.

        Args:
            line_ (object): A Circuit object being updated.
            value (dict): A dictionary containing the direct mapping configuration.
            row (pd.Series): A row from the GeoDataFrame containing circuit-related data.

        This method processes the direct mapping configuration by iterating through the
        key-value pairs of the 'value' dictionary and directly setting the corresponding
        attribute on the Circuit object using the value from the row.
        """
        for mapping_key, mapping_value in value.items():
            setattr(line_, f"_{mapping_key}", row[mapping_value])

    @staticmethod
    def _process_indirect_mapping(line_, value, row):
        """
        Static method to process the indirect mapping configuration for a Circuit object.

        Args:
            line_ (object): A Circuit object being updated.
            value (dict): A dictionary containing the indirect mapping configuration.
            row (pd.Series): A row from the GeoDataFrame containing circuit-related data.

        This method processes the indirect mapping configuration by iterating through the
        key-value pairs of the 'value' dictionary. If the value is a list, it treats the
        first element as a parameter name and the second element as a function name. The
        method then retrieves the parameter value from the row and calls the specified
        function with that parameter value. The result is then set as an attribute on the
        Circuit object.

        If the value is not a list, the method directly sets the corresponding attribute on
        the Circuit object using the value from the row.
        """
        for mapping_key, mapping_value in value.items():
            if isinstance(mapping_value, list):
                param_name, function_name = mapping_value
                function_ = globals()[function_name]
                param_value = row[param_name]
                setattr(line_, f"_{mapping_key}", function_(str(param_value)))        # corrigingo para string para encontrar valor no dicionario
            else:
                setattr(line_, f"_{mapping_key}", row[mapping_value])

    @staticmethod
    def _process_calculated(line_, value, row):
        """
        Static method to process the calculated mapping configuration for a Line object.

        Args:
            circuit_ (object): A Circuit object being updated.
            value (dict): A dictionary containing the direct mapping configuration.
            row (pd.Series): A row from the GeoDataFrame containing circuit-related data.

        This method processes the direct mapping configuration by iterating through the
        key-value pairs of the 'value' dictionary and directly setting the corresponding
        attribute on the Circuit object using the value from the row.
        """
        for mapping_key, mapping_value in value.items():
            
            expression = ""
            for item in mapping_value:
                if isinstance(item, str) and any(char.isalpha() for char in item):
                    print(row[item])
                    expression = f'{expression} {row[item]}'
                else:
                    expression = f'{expression}{item}'

            param_value = eval(expression)
            print(param_value)
            setattr(line_, f"_{mapping_key}", param_value)


    @staticmethod
    def _create_line_from_row(line_config, row):
        line_ = Line()

        for key, value in line_config.items():
            if key == "static":
                line_._process_static(line_, value)
            elif key == "direct_mapping":
                line_._process_direct_mapping(line_, value,row)
            elif key == "indirect_mapping":
                line_._process_indirect_mapping(line_, value,row)
            elif key == "calculated":
                line_._process_calculated(line_, value, row)

        return line_


    @staticmethod
    def create_line_from_json(json_data: Any, dataframe: gpd.geodataframe.GeoDataFrame):
        lines = []
        line_config = json_data['elements']['Line']['SSDMT']
        print(line_config)
        print(dataframe)
        progress_bar = tqdm(dataframe.iterrows(), total=len(dataframe), desc="Line", unit=" lines", ncols=100)
        for _, row in progress_bar:
            line_ = Line._create_line_from_row(line_config, row)
            lines.append(line_)
            # progress_bar.set_description(f"Processing Line {_ + 1}")

        return lines


In [ ]:


lines = []
line_config = json_data.data['elements']['Line']['SSDMT']
line_config


In [93]:
line_ = Line()
lines = Line.create_line_from_json(json_data.data, case.dfs['SSDMT']['gdf'])

{'arquivo': 'Trecho_MT', 'static': {'units': 'km'}, 'direct_mapping': {'bus1': 'PAC_1', 'bus2': 'PAC_2', 'line': 'COD_ID', 'linecode': 'TIP_CND'}, 'indirect_mapping': {'phases': ['FAS_CON', 'convert_tfascon_phases']}, 'calculated': {'length': ['COMP', '/', 1000]}}
     COD_ID      CTMT  PAC_1   PAC_2 FAS_CON  TIP_CND   COMP
0      2446  2_TAP2_1  86751   86761     ABC       56  32.66
1      2447  2_TAP2_1  86761  1CF155     ABC       56  15.07
2      2448  2_TAP2_1  86771   10931     ABC       70  48.72
3      2452  2_TAP2_1  86921   27121     ABC        3  39.81
4      2449  1_TAP2_1   2371   86851     ABC       70  46.28
...     ...       ...    ...     ...     ...      ...    ...
2064     68  1_TAP2_1  21921   21931     ABC       70   3.00
2065     69  1_TAP2_1   7911   1CS16     ABC       67  39.50
2066     70  1_TAP2_1  29291   33171     ABC       67   7.50
2067     71  1_TAP2_1   2351    2361     ABC       70  38.00
2068     72  1_TAP2_1   1471   34601     ABC       63  20.00

[2

Line:  28%|█████████████▌                                  | 582/2069 [00:00<00:00, 5819.63 lines/s]

32.66
0.032659999999999995
15.07
0.01507
48.72
0.04872
39.81
0.039810000000000005
46.28
0.04628
37.24
0.03724
41.19
0.04119
31.01
0.031010000000000003
27.57
0.02757
16.45
0.01645
28.22
0.02822
13.72
0.013720000000000001
23.95
0.02395
31.09
0.03109
43.91
0.04391
3.52
0.00352
4.17
0.00417
44.5
0.0445
31.78
0.03178
27.01
0.027010000000000003
38.3
0.038299999999999994
33.48
0.033479999999999996
36.28
0.03628
49.86
0.04986
23.64
0.02364
27.41
0.02741
15.08
0.01508
31.94
0.03194
12.59
0.01259
74.75
0.07475
89.26
0.08926
18.04
0.01804
8.9
0.0089
37.84
0.037840000000000006
39.87
0.039869999999999996
15.37
0.01537
19.04
0.019039999999999998
18.23
0.01823
39.13
0.039130000000000005
53.5
0.0535
37.0
0.037
71.0
0.071
20.07
0.02007
10.62
0.01062
34.84
0.03484
12.86
0.01286
30.14
0.03014
7.74
0.00774
28.51
0.02851
48.5
0.0485
42.29
0.04229
19.93
0.01993
87.0
0.087
86.64
0.08664
40.36
0.04036
47.14
0.04714
45.98
0.04598
22.56
0.02256
29.26
0.02926
19.98
0.01998
34.86
0.03486
29.14
0.02914
16.55
0.016

Line: 100%|███████████████████████████████████████████████| 2069/2069 [00:00<00:00, 7109.53 lines/s]

21.05
0.02105
54.0
0.054
4.2
0.004200000000000001
26.28
0.02628
32.91
0.032909999999999995
78.44
0.07844
15.15
0.01515
77.0
0.077
38.5
0.0385
17.72
0.01772
45.97
0.04597
47.0
0.047
46.72
0.04672
32.19
0.032189999999999996
17.42
0.01742
16.0
0.016
40.0
0.04
39.0
0.039
32.0
0.032
40.78
0.040780000000000004
48.14
0.04814
25.5
0.0255
41.0
0.041
40.0
0.04
41.5
0.0415
18.0
0.018
41.5
0.0415
40.0
0.04
37.0
0.037
44.0
0.044
5.88
0.00588
43.82
0.04382
41.0
0.041
69.44
0.06944
27.57
0.02757
37.0
0.037
8.39
0.00839
48.01
0.04801
4.76
0.0047599999999999995
42.0
0.042
43.0
0.043
69.98
0.06998
23.0
0.023
18.13
0.01813
33.63
0.03363
39.0
0.039
36.0
0.036
37.0
0.037
31.74
0.03174
21.22
0.02122
15.28
0.01528
36.0
0.036
34.5
0.0345
42.0
0.042
47.95
0.04795
6.46
0.00646
25.0
0.025
79.0
0.079
32.15
0.03215
39.0
0.039
81.21
0.08120999999999999
17.0
0.017
43.0
0.043
41.0
0.041
84.74
0.08474
15.16
0.01516
77.38
0.07737999999999999
12.5
0.0125
33.0
0.033
90.16
0.09015999999999999
41.0
0.041
5.0
0.005
38.0
0.0

In [85]:
line_.get("_units")

AttributeError: 'Line' object has no attribute 'get'

In [46]:
line = Line()


In [ ]:
line_config

In [81]:
lines = []
line_config = json_data.data['elements']['Line']['SSDMT']
dataframe = case.dfs['SSDMT']['gdf']


progress_bar = tqdm(dataframe.iterrows(), total=len(dataframe), desc="Line", unit=" lines", ncols=100)
for _, row in progress_bar:
        
        line_ = Line._create_line_from_row(line_config, row)
        
        lines.append(line_)
        progress_bar.set_description(f"Processing Line {_ + 1}")

# lines

Processing Line 144:   3%|█▏                                 | 72/2069 [00:00<00:02, 719.96 lines/s]

32.66
0.032659999999999995
15.07
0.01507
48.72
0.04872
39.81
0.039810000000000005
46.28
0.04628
37.24
0.03724
41.19
0.04119
31.01
0.031010000000000003
27.57
0.02757
16.45
0.01645
28.22
0.02822
13.72
0.013720000000000001
23.95
0.02395
31.09
0.03109
43.91
0.04391
3.52
0.00352
4.17
0.00417
44.5
0.0445
31.78
0.03178
27.01
0.027010000000000003
38.3
0.038299999999999994
33.48
0.033479999999999996
36.28
0.03628
49.86
0.04986
23.64
0.02364
27.41
0.02741
15.08
0.01508
31.94
0.03194
12.59
0.01259
74.75
0.07475
89.26
0.08926
18.04
0.01804
8.9
0.0089
37.84
0.037840000000000006
39.87
0.039869999999999996
15.37
0.01537
19.04
0.019039999999999998
18.23
0.01823
39.13
0.039130000000000005
53.5
0.0535
37.0
0.037
71.0
0.071
20.07
0.02007
10.62
0.01062
34.84
0.03484
12.86
0.01286
30.14
0.03014
7.74
0.00774
28.51
0.02851
48.5
0.0485
42.29
0.04229
19.93
0.01993
87.0
0.087
86.64
0.08664
40.36
0.04036
47.14
0.04714
45.98
0.04598
22.56
0.02256
29.26
0.02926
19.98
0.01998
34.86
0.03486
29.14
0.02914
16.55
0.016

Processing Line 315:  12%|████                              | 248/2069 [00:00<00:02, 842.77 lines/s]

31.0
0.031
21.0
0.021
28.19
0.02819
37.0
0.037
41.79
0.04179
36.0
0.036
40.53
0.040530000000000004
40.32
0.04032
39.96
0.03996
42.93
0.04293
52.65
0.052649999999999995
38.25
0.03825
41.01
0.04101
40.82
0.04082
41.27
0.04127
42.57
0.042570000000000004
39.36
0.03936
40.74
0.040740000000000005
43.14
0.04314
39.0
0.039
41.11
0.04111
41.0
0.041
42.42
0.04242
1.59
0.00159
15.82
0.01582
35.1
0.0351
29.89
0.02989
96.73
0.09673000000000001
39.0
0.039
42.0
0.042
41.9
0.0419
47.0
0.047
17.08
0.017079999999999998
20.35
0.02035
38.77
0.038770000000000006
20.87
0.02087
22.37
0.02237
41.21
0.041210000000000004
38.69
0.038689999999999995
31.78
0.03178
39.09
0.03909000000000001
74.12
0.07412
44.0
0.044
40.0
0.04
40.0
0.04
60.03
0.06003
39.34
0.03934
6.0
0.006
38.55
0.038549999999999994
38.48
0.03848
38.39
0.03839
38.27
0.038270000000000005
38.28
0.03828
38.97
0.03897
38.18
0.03818
34.88
0.03488
29.95
0.02995
40.91
0.040909999999999995
40.43
0.04043
41.12
0.04112
41.53
0.041530000000000004
37.64
0.03764

Processing Line 486:  21%|███████▏                          | 439/2069 [00:00<00:01, 910.97 lines/s]

...

32.0
0.032
41.0
0.041
36.94
0.03694
31.16
0.03116
41.0
0.041
43.0
0.043
4.1
0.0040999999999999995
46.09
0.046090000000000006
46.14
0.04614
43.0
0.043
29.46
0.02946
23.66
0.02366
82.55
0.08255
30.9
0.030899999999999997
47.46
0.04746
28.23
0.02823
26.78
0.02678
25.09
0.02509
19.0
0.019
37.95
0.037950000000000005
18.67
0.018670000000000003
40.0
0.04
37.43
0.03743
40.0
0.04
35.0
0.035
33.0
0.033
40.0
0.04
42.5
0.0425
10.39
0.01039
45.18
0.04518
33.0
0.033
58.72
0.05872
35.0
0.035
33.0
0.033
42.1
0.0421
50.0
0.05
26.5
0.0265
42.0
0.042
42.0
0.042
69.24
0.06924
18.56
0.01856
36.27
0.036270000000000004
47.0
0.047
31.09
0.03109
32.07
0.03207
37.0
0.037
46.0
0.046
45.0
0.045
42.0
0.042
41.62
0.04162
81.55
0.08155
27.56
0.027559999999999998
49.0
0.049
21.79
0.02179
77.43
0.07743000000000001
38.5
0.0385
46.0
0.046
86.0
0.086
37.0
0.037
39.0
0.039
47.0
0.047
79.6
0.07959999999999999
32.0
0.032
37.0
0.037
39.0
0.039
46.0
0.046
43.0
0.043
26.87
0.02687
22.34
0.02234
29.0
0.029
32.0
0.032
70.67
0.070

Processing Line 1942:  91%|█████████████████████████████▎  | 1893/2069 [00:02<00:00, 909.93 lines/s]

16.0
0.016
42.0
0.042
36.14
0.03614
97.0
0.097
39.0
0.039
10.0
0.01
38.0
0.038
38.01
0.038009999999999995
80.81
0.08081
40.5
0.0405
16.05
0.016050000000000002
35.0
0.035
75.31
0.07531
43.88
0.04388
79.0
0.079
20.41
0.02041
24.0
0.024
35.0
0.035
38.0
0.038
68.02
0.06802
71.56
0.07156
44.0
0.044
86.0
0.086
84.83
0.08483
79.97
0.07997
5.62
0.00562
91.49
0.09148999999999999
32.0
0.032
86.8
0.0868
98.91
0.09891
75.86
0.07586
41.0
0.041
94.03
0.09403
68.6
0.0686
45.0
0.045
34.83
0.03483
42.0
0.042
25.53
0.02553
49.97
0.04997
39.0
0.039
22.78
0.02278
39.0
0.039
42.5
0.0425
36.0
0.036
45.0
0.045
33.15
0.03315
37.0
0.037
75.38
0.07537999999999999
9.0
0.009
40.0
0.04
40.0
0.04
48.0
0.048
22.0
0.022
41.0
0.041
27.0
0.027
95.0
0.095
79.0
0.079
8.0
0.008
20.76
0.02076
35.03
0.03503
33.0
0.033
11.34
0.01134
76.79
0.07679000000000001
46.0
0.046
10.0
0.01
40.0
0.04
14.0
0.014
73.61
0.07361
24.64
0.024640000000000002
70.91
0.07091
37.5
0.0375
70.32
0.07032
39.0
0.039
88.96
0.08896
42.0
0.042
53.0
0.053

Processing Line 2069: 100%|████████████████████████████████| 2069/2069 [00:02<00:00, 857.38 lines/s]

16.46
0.016460000000000002
41.23
0.041229999999999996
34.0
0.034
27.02
0.02702
35.0
0.035
88.13
0.08813
37.0
0.037
40.0
0.04
26.01
0.026010000000000002
40.0
0.04
25.18
0.02518
39.0
0.039
39.0
0.039
47.0
0.047
46.0
0.046
41.0
0.041
40.0
0.04
61.0
0.061
31.55
0.03155
26.0
0.026
39.0
0.039
44.0
0.044
76.51
0.07651000000000001
80.0
0.08
46.0
0.046
39.0
0.039
35.01
0.03501
80.74
0.08073999999999999
31.0
0.031
39.95
0.03995
43.0
0.043
46.0
0.046
38.0
0.038
41.0
0.041
40.0
0.04
70.32
0.07032
83.0
0.083
38.99
0.038990000000000004
37.0
0.037
79.79
0.07979
90.01
0.09001
5.29
0.00529
78.24
0.07823999999999999
79.84
0.07984000000000001
89.64
0.08964
41.8
0.0418
80.33
0.08033
26.02
0.026019999999999998
38.0
0.038
71.0
0.071
30.0
0.03
39.0
0.039
39.0
0.039
23.0
0.023
50.0
0.05
37.0
0.037
37.0
0.037
15.0
0.015
45.79
0.04579
18.0
0.018
33.05
0.033049999999999996
40.0
0.04
35.0
0.035
13.08
0.01308
34.0
0.034
37.0
0.037
40.0
0.04
42.0
0.042
38.0
0.038
80.58
0.08058
37.0
0.037
37.46
0.03746
24.86
0.02486

In [82]:
lines

[New "Line.SMT_2446" phases=3 bus1="86751.1.2.3" bus2="86761.1.2.3" linecode="56_3" length=0.03266 units=km,
 New "Line.SMT_2447" phases=3 bus1="86761.1.2.3" bus2="1CF155.1.2.3" linecode="56_3" length=0.01507 units=km,
 New "Line.SMT_2448" phases=3 bus1="86771.1.2.3" bus2="10931.1.2.3" linecode="70_3" length=0.04872 units=km,
 New "Line.SMT_2452" phases=3 bus1="86921.1.2.3" bus2="27121.1.2.3" linecode="3_3" length=0.03981 units=km,
 New "Line.SMT_2449" phases=3 bus1="2371.1.2.3" bus2="86851.1.2.3" linecode="70_3" length=0.04628 units=km,
 New "Line.SMT_2450" phases=3 bus1="86851.1.2.3" bus2="2391.1.2.3" linecode="70_3" length=0.03724 units=km,
 New "Line.SMT_2451" phases=3 bus1="25191.1.2.3" bus2="86921.1.2.3" linecode="3_3" length=0.04119 units=km,
 New "Line.SMT_2430" phases=3 bus1="86301.1.2.3" bus2="86291.1.2.3" linecode="114_3" length=0.03101 units=km,
 New "Line.SMT_2431" phases=3 bus1="86311.1.2.3" bus2="86301.1.2.3" linecode="114_3" length=0.02757 units=km,
 New "Line.SMT_2432"

In [1]:
line_config = json_data.data['elements']['Line']['SSDMT']
line_config

dataframe = case.dfs['SSDMT']['gdf']
dataframe
dataframe_code =  case.dfs['SEGCON']['gdf']
dataframe_code


NameError: name 'json_data' is not defined

In [253]:
line_config

{'arquivo': 'Trecho_MT',
 'static': {'units': 'km'},
 'direct_mapping': {'bus1': 'PAC_1',
  'bus2': 'PAC_2',
  'line': 'COD_ID',
  'linecode': 'TIP_CND'},
 'indirect_mapping': {'phases': ['FAS_CON', 'convert_tfascon_phases']},
 'calculated': {'length': ['COMP', '/', 100]}}

In [136]:
name = dataframe.iloc[0]["COD_ID"]

for key, value in dataframe.items():

    print(key, value[1])

COD_ID 2447
CTMT 2_TAP2_1
PAC_1 86761
PAC_2 1CF155
FAS_CON ABC
TIP_CND 56
COMP 15.07


In [290]:
lines

[New "Line.2446" phases=3 bus1="86751" bus2="86761" linecode="56" length=0.00000 units=km,
 New "Line.2447" phases=3 bus1="86761" bus2="1CF155" linecode="56" length=0.00000 units=km,
 New "Line.2448" phases=3 bus1="86771" bus2="10931" linecode="70" length=0.00000 units=km,
 New "Line.2452" phases=3 bus1="86921" bus2="27121" linecode="3" length=0.00000 units=km,
 New "Line.2449" phases=3 bus1="2371" bus2="86851" linecode="70" length=0.00000 units=km,
 New "Line.2450" phases=3 bus1="86851" bus2="2391" linecode="70" length=0.00000 units=km,
 New "Line.2451" phases=3 bus1="25191" bus2="86921" linecode="3" length=0.00000 units=km,
 New "Line.2430" phases=3 bus1="86301" bus2="86291" linecode="114" length=0.00000 units=km,
 New "Line.2431" phases=3 bus1="86311" bus2="86301" linecode="114" length=0.00000 units=km,
 New "Line.2432" phases=3 bus1="86321" bus2="86411" linecode="114" length=0.00000 units=km,
 New "Line.2433" phases=3 bus1="86331" bus2="24751" linecode="56" length=0.00000 units=km,

In [285]:
def rename_linecde_string(input_str: str) -> str:
        
        """
        This function re-writes the string identfying key places by specified parameters and insering caracteres.  
        Precisa receber um argumento para definir SMT ou outra entidade. 

        Args: input_str, datagrame para identificar a entidade

        Returns: string modified

        In this case, it should modify the names of line, bus1, bus2 and linecode. 

        """

        pattern = r'New "Line.([\dA-Za-z]+)" phases=(\d+) bus1="([\dA-Za-z]+)" bus2="([\dA-Za-z]+)" linecode="(\d+)"'   #linecode=(\d+) length=(\d+) units=(\d+)' # bus1="86761" bus2="1CF155" linecode="56"'

        def repl(match):
            line_num = match.group(1)
            phases = match.group(2)
            bus1 = match.group(3)     
            bus2 = match.group(4)       
            linecode = match.group(5)
            return f'New "Line.SMT_{line_num}" phases={phases} bus1="{bus1}.1.2.3" bus2="{bus2}.1.2.3" linecode="{linecode}_{phases}"'

        return re.sub(pattern, repl, input_str)

In [ ]:
CTMT 2_TAP2_1
PAC_1 86761
PAC_2 1CF155
FAS_CON ABC
TIP_CND 56
COMP 15.07

In [230]:
line = "24C47"
phases = 3
bus1 = 86761
bus2 = "1CF155"
linecode = 56
length = 15.07
units = "km"

In [287]:
stringerss = f'New "Line.{line}" phases={phases} ' \
                f'bus1="{bus1}" bus2="{bus2}" ' \
                f'linecode="{linecode}" length={length:.5f} ' \
                f'units={units}'

stringerss = f'New "Line.2446" phases=3 bus1="86751" bus2="86761" linecode="56" length=0.00000 units=km'
print(stringerss)


New "Line.2446" phases=3 bus1="86751" bus2="86761" linecode="56" length=0.00000 units=km


In [289]:
testando3 = rename_linecode_string(stringerss)
testando3

'New "Line.SMT_2446" phases=3 bus1="86751.1.2.3" bus2="86761.1.2.3" linecode="56_3" length=0.00000 units=km'

In [213]:
import re

def rename_linecode_string2(input_str: str) -> str:
    """
    This function re-writes the string identifying key places by specified parameters and inserting characters.
    Precisa receber um argumento para definir SMT ou outra entidade.

    Args:
        input_str (str): The input string to be modified.
        Dataframe para identificar a entidade BDGD.

    Returns:
        str: The modified string.

    In this case, it should modify the names of line, bus1, bus2, and linecode.
    """

    pattern = r'New "Line.(\d+)" phases=(\d+) bus1="(\d+)" bus2="(\d+)"'

    def repl(match):
        line_num = match.group(1)
        phases = match.group(2)
        bus1 = match.group(3)
        bus2 = match.group(4)

    
        return f'New "Line.SMT_{line_num}" phases={phases} bus1="{bus1}.1.2.3" bus2="{bus2}.1.2.3"'

    return re.sub(pattern, repl, input_str)

# Example usage
input_str = 'New "Line.123" phases=2 bus1="456" bus2="789"'
modified_str = rename_linecode_string(input_str)
print(modified_str)


New "Line.SMT_123" phases=2 bus1="456.1.2.3" bus2="789.1.2.3"
